In [3]:
# https://huggingface.co/spaces/MultiTransformer/autogen-tutorials/blob/main/agentchat_RetrieveChat.ipynb

In [4]:
import autogen
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
import chromadb

In [5]:
def termination_msg(msg):
        content = msg.get("content")
        if content.lower() in ("exit", "quit", "stop", "done", "terminate"):
            return True
        return False

In [6]:
model = 'gpt-3.5-turbo-0125'
config_file="CONFIG_LIST.json"
filter_dict = {"model": [model]}
config_list = autogen.config_list_from_json(
                        env_or_file=config_file,
                        filter_dict=filter_dict,
                    )

In [7]:
llm_config = {
    "functions": [
        {
            "name": "retrieve_content",
            "description": "retrieve content of the linux kernel source code and documantation for code generation and question answering.",
            "parameters": {
                "type": "object",
                "properties": {
                    "message": {
                        "type": "string",
                        "description": "retrieve content of the linux kernel source code and documantation for code generation and question answering.",
                    }
                },
                "required": ["message"],
            },
        },
    ],
    "config_list": config_list,
    "timeout": 120,
    "seed": 50,
}

manager_llm_config = {
    "config_list": config_list,
    "timeout": 120,
    "seed": 50,
}

In [8]:
rag_path="rag/Linux/"
persist_directory = "/tmp/chromadb"
prompt_file1 = "prompts\KernelGuru.txt"
prompt_file2 = "prompts\PatchPro.txt"
task_file = "prompts/task.txt"
PROMPT1 = open(prompt_file1).read()
PROMPT2 = open(prompt_file1).read()
task = open(task_file).read()

In [9]:
chroma_client = chromadb.Client()

In [10]:
# Create a new collection for NaturalQuestions dataset
# `task` indicates the kind of task we're working on. In this example, it's a `qa` task.
ragproxyagent = RetrieveUserProxyAgent(
    is_termination_msg=termination_msg,
    name="ragproxyagent",
    system_message="""Assistant who has extra content retrieval power with the linux kernel source code and documantation for solving difficult problems.""",
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    # max_consecutive_auto_reply=10,
    retrieve_config={
        "task": "qa",
        "docs_path": rag_path,
        "chunk_token_size": 2048,
        "model": model,
        "client": chroma_client,
        "collection_name": "rag_collection",
        "chunk_mode": "one_line",
        "embedding_model": "all-MiniLM-L6-v2",
        # "customized_prompt": PROMPT,
        # "human_input_mode": "ALWAYS",
    },
)

In [11]:
def retrieve_content(message, n_results=3):
        ragproxyagent.n_results = n_results  # Set the number of results to be retrieved.
        # Check if we need to update the context.
        update_context_case1, update_context_case2 = ragproxyagent._check_update_context(message)
        if (update_context_case1 or update_context_case2) and ragproxyagent.update_context:
            ragproxyagent.problem = message if not hasattr(ragproxyagent, "problem") else ragproxyagent.problem
            _, ret_msg = ragproxyagent._generate_retrieve_user_reply(message)
        else:
            ret_msg = ragproxyagent.generate_init_message(message, n_results=n_results)
        return ret_msg if ret_msg else message

In [12]:
# from autogen.agentchat.contrib.agent_builder import AgentBuilder

# builder = AgentBuilder(
#                     config_file_or_env=config_file,
#                     builder_model=model,
#                     agent_model=model)
# agent_list, agent_configs = builder.build(PROMPT, llm_config, coding=True)
# agent_list.append(ragproxyagent)

In [13]:
# def start_task(execution_task: str, agent_list: list, llm_config: dict):
#     group_chat = autogen.GroupChat(agents=agent_list,
#                                     messages=[],
#                                     max_round=12
#                                 )
#     manager = autogen.GroupChatManager(
#                             groupchat=group_chat,
#                             llm_config=llm_config
#             )
#     agent_list[0].initiate_chat(manager, message=execution_task)

# start_task(
#     execution_task=task,
#     agent_list=agent_list,
#     llm_config=llm_config,
# )

In [14]:
agent1 = RetrieveAssistantAgent(
    name="Kernel_Guru",
    system_message=PROMPT1,
    llm_config=llm_config,
    human_input_mode="TERMINATE",
    is_termination_msg=termination_msg,
)

In [15]:
agent1 = RetrieveAssistantAgent(
    name="Kernel_Guru",
    system_message=PROMPT1,
    llm_config=llm_config,
)

In [16]:
agent2 = RetrieveAssistantAgent(
    name="Patch_Pro",
    system_message=PROMPT2,
    llm_config=llm_config,
    human_input_mode="TERMINATE",
    is_termination_msg=termination_msg,
)

In [17]:
for agent in [agent1, agent2]:
    # register functions for all agents.
    agent.register_function(
        function_map={
            "retrieve_content": retrieve_content,
        }
    )

In [19]:
agent1.reset()
agent2.reset()
group_chat = autogen.GroupChat(agents=[agent1, agent2],
                                messages=[],
                                max_round=12,
                                speaker_selection_method="round_robin",
                            )
manager = autogen.GroupChatManager(
                        groupchat=group_chat,
                        llm_config=manager_llm_config
        )

In [20]:
manager.initiate_chat(manager,
                      message=task,
                    )

chat_manager (to chat_manager):

Please help to resolve issue in the linux kernel.
Bug 218514
Summary:	iwlwifi null pointer dereference when debugfs=off
Product:	Drivers	Reporter:	drastic_endurance425
Component:	network-wireless-intel	Assignee:	Default virtual assignee for network-wireless-intel (drivers_network-wireless-intel)
Status:	NEW ---	 	 
Severity:	normal	CC:	drastic_endurance425
Priority:	P3	 	 
Hardware:	All	 	 
OS:	Linux	 	 
Kernel Version:	
Regression:	No
Attachments:	journalctl log
Description: drastic_endurance425 2024-02-20 14:47:42 UTC
Created attachment 305900 [details]
journalctl log
Some commit part of the 6.7 kernel series has resulted in the bootup sequence to issue me an oops. One of my kernel parameters is debugfs=off and NetworkManager exits with a kernel NULL pointer dereference. This issue is present whether I use the vanilla kernel 6.7.5 or the zen kernel. I attached my journalctl
what files are involved in the above issue?
What fix should be applied to solv

d:\miniforge3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


doc_ids:  [['doc_73956', 'doc_162492', 'doc_73988']]
Adding doc_id doc_73956 to context.
Adding doc_id doc_162492 to context.
Adding doc_id doc_73988 to context.
Patch_Pro (to chat_manager):

***** Response from calling function "retrieve_content" *****
You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
You must give as short an answer as possible.

User's question is: Please provide the relevant files and documentation related to the iwlwifi null pointer dereference issue when debugfs=off (Bug 218514)

Context is: │   │   ├── trace_functions.c
│   │   ├── trace_functions.c
│   │   ├── tracing_map.c


*************************************************************

--------------------------------------------------------------------------------
Kernel_Guru (to chat_manager):

UPDATE CONTEXT

------